# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

#### Загружаем данные и выводим первые 10 строк на экран

In [1]:
import pandas as pd
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**При просмотре данных появились следующие комментарии:**

- Колличество отработанных дней в колонке `days_employed` часто имеет отрицательное значение, так же встречаются пропуски и положительные значения.

- В колонке `education` для написания одних и тех же значений используется разный региср, например **ВЫСШЕЕ/высшее** или **СРЕДНЕЕ/Среднее/среднее**.

- В колонке `purpose` одинаковы цели могут быть написаны по-разному, например **покупка жилья/покупка жилья для семьи/операции с жильем** и т.д.

#### Посмотрим информацию о таблице командой `info()`

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски есть в колонках `days_employed` и `total_income`, причем их количество одинаковое. <br>
Проверим совпадают ли они:

In [3]:
(data['days_employed'].isna()==data['total_income'].isna()).unique()

array([ True])

Везде, где не указано количиство отработанных дней, нет информации о доходе. Пропуски в колонках `days_employed` и `total_income` совпадают

### Шаг 2.1 Заполнение пропусков

#### Посчитаем количество пропусков в столбце `total_income`

In [4]:
data['total_income'].isnull().sum()

2174

####  Количество пропусков
В колонках `days_employed` и `total_income` по 2174 пропуска. Возможно, те граждане, у которых не указаны данные брали небольшие суммы или брали кредит под залог чего-то, поэтому не требовалось указывать информацию о работе и доходе. К сожалению такой информации в таблице нет. Или просто операторы по какой-то причине не внесли данные.


#### Заполнение пропусков в колонке с доходами медианым значением
Заполним пропуски в доходах медианным значением. Оно лучше всего описывает наиболее часто встречающиеся значения. Среднее значение хуже, потому что появление в выборки нескольких человек с высоким доходом существенно повысит среднее значение.<br>
Посчитаем медианное значение дохода:

In [5]:
income_median = data['total_income'].median()
income_median

145017.93753253992

Заполним пропуске в столбце `total_income` медианным значением и посчитаем количество пустых значений после заполнения

In [6]:
data['total_income'] = data['total_income'].fillna(income_median)
data['total_income'].isnull().sum()

0

Пустых значений больше нет.

#### Отрицательные значения в колонке `days_employed`
В столбце `days_employed` есть отрицательные значения, посчитаем сколько их, а так же сколько положительных и пустых значений:

In [7]:
display(data[data['days_employed']<0]['days_employed'].count()) # Количество отрицательных значений
display(data[data['days_employed']>0]['days_employed'].count()) # Количество положительных значений
data['days_employed'].isnull().sum() # Количество пустых значений

15906

3445

2174

#### Возможные причины отрицательных значений
Отрицательных значений намного больше, чем положительных, вряд ли это опечатка. Возможно в части банков в системе момент подачи заявки на кредит расматривается как 0, а количесвто отработанных дней на момент подачи заявки получается отрицаетльным.

#### Перевод отрицательных значений в положительные
Возьмем модуль от столбца и проверим, что все отрицательные значения стали положительными, посмотрев на первые 10 строк таблицы

In [8]:
data['days_employed'] = data['days_employed'].abs()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Заполнение пропусков в колонке `days_employed` медианными значениями
Вычислим медианое значение и заполним им пропуски. По той же причине, что и в столбце с доходами, медиана лучше подходит для описания времени работы для большинства людей. Несколько высоких значений существенно сдвигают среднее в область больших значений, поэтому среднее хуже описывает наиболее типичную продолжительность работы.

In [9]:
days_employed_median = data['days_employed'].median() #Вычисляем медиану
display(days_employed_median) # Выводим медиану на экран
data['days_employed'] = data['days_employed'].fillna(days_employed_median) # Заполняем пропуски медианой
data['days_employed'].isnull().sum() #Проверяем, что пропусков больше нет


2194.220566878695

0

### Шаг 2.2 Проверка данных на аномалии и исправления.

#### Приведение к единому регистру
Чтобы корректно обработать дубликаты приведем значения в столбце `education` к единому регистру
Сначала посмотрим какие значения есть в столбце

In [10]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем все к нижнему регистру и проверим результат

In [11]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим колонку `family_status`

In [12]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Все статусы начинаются с маленькой буквы, а 'Не женат / не замужем' с большой, приведем все к единому виду

In [13]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

#### Колонка `children`

Посмотрим какие значения есть в колонеке `children`

In [14]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Встречаются значения -1. Исправим это значение на 0. Есть значение 20. Cкорее всего это опечатка и имелось ввиду 2 ребенка. Но проверить это мы не можем, и таких значений относительно немного, поэтому лучше удалим их.

In [15]:
data.loc[data['children']==-1,'children'] = 0
data['children'].value_counts()

0     14196
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [16]:
data = data.drop(data[data['children']==20].index)
data['children'].value_counts()

0    14196
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

#### Колонка `gender`

In [17]:
data['gender'].value_counts()

F      14189
M       7259
XNA        1
Name: gender, dtype: int64

В одной строке есть неопределившийся с полом, такие нам не нужны, по крайней мере, если мы никуда не собираемся уезжать.

In [18]:
data = data.drop(data[data['gender']=='XNA'].index)
data['gender'].value_counts()

F    14189
M     7259
Name: gender, dtype: int64

### Шаг 2.3. Изменение типов данных.

Заменим тип данных в столбце `total_income` на целочисленный и проверим результат

In [19]:
data['total_income'] = data['total_income'].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21448 non-null  int64  
 1   days_employed     21448 non-null  float64
 2   dob_years         21448 non-null  int64  
 3   education         21448 non-null  object 
 4   education_id      21448 non-null  int64  
 5   family_status     21448 non-null  object 
 6   family_status_id  21448 non-null  int64  
 7   gender            21448 non-null  object 
 8   income_type       21448 non-null  object 
 9   debt              21448 non-null  int64  
 10  total_income      21448 non-null  int64  
 11  purpose           21448 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

#### Удаление явных дубликатов

Псмотрим есть ли в таблице явные дубликаты

In [20]:
data.duplicated().sum()

71

После приведения столбца `eduction` к единому регистру в таблице найден 71 явный дубликат.  Избавимся от них методом `drop_duplicates()`, перезапишем индексы попрядку и проверим результат.

In [21]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Дубликаты появились только после приведения к единому регистру и их не так много, возможно во время занесения данных возникали какие-то технические неполадки, и данные внесли повторно, слегка изменив их.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим датафрейм `education_df`, где каждому уникальному значению из `education` соответствует уникальное значение `education_id`

In [22]:
education_df = data[['education_id','education']]
education_df.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


Посмотрим какой id соответствует каждому типу образования, удалим все дубликаты.

In [23]:
education_df = education_df.drop_duplicates().reset_index(drop=True)
education_df

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Проделаем тже самое для семейного статуса. Создадим датафрейм `family_status_df`, где каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id`. 

In [24]:
family_status_df = data[['family_status_id','family_status']]
family_status_df.head()

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак


Посмотрим соответсвие семейного статуса и категории. Удалим дубликаты.

In [25]:
family_status_df = family_status_df.drop_duplicates().reset_index(drop=True)
family_status_df

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Удалим колонки `education` и `family_status` из исходной таблицы.

In [26]:
data = data.drop(columns=['family_status','education'])
data.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### Шаг 2.6. Категоризация дохода.

Опишем функцию которая возвращает категорию в зависимости от дохода
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.
И проверим как она работает

In [27]:
def income_category(income):
    if 0<=income<=30000:
        return 'E'
    elif 30001<=income<=50000:
        return 'D'
    elif 50001<=income<=200000:
        return 'C'
    elif 200001<=income<1000000:
        return 'B'
    else:
        return 'A'
display(income_category(20000))
income_category(50001)

'E'

'C'

Создадим столбец `total_income_category` методом `apply()` и посмотрим шапку.

In [28]:
data['total_income_category'] = data['total_income'].apply(income_category)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Интересно, сколько человек получают больше 1000000 рублей в месяц

In [29]:
data['total_income_category'].value_counts()

C    15960
B     5020
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

Напишем функцию, которая будет возвращать категорию кредита
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Но сначала посмотрим на уникальные значения в столбце `purpose`

In [30]:
data['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     764
сыграть свадьбу                           761
операции с недвижимостью                  674
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   650
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          623
покупка своего жилья                      620
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

Жилье и недвижимость попадают в одну категорию 'операции с недвижимостью'. Бидем проверять, чтобы в строке было 'жиль' или 'недвижим'.<br>
В 'операции с автомобилем' попадут строки, где есть 'автомоб'.<br>
В 'проведение свадьбы', где есть 'свадьб'.<br>
В 'получение образования', где есть 'образован'





In [31]:
def purpose_category(purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif 'жиль' in purpose or 'недвижим' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    else:
        return 'unknown'
    

Применим полученную функцию к столбцу `purpose`

In [32]:
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Посмотрим как она сработала, заменила ли она все значения на нужные категории?

In [33]:
data['purpose_category'].value_counts()

операции с недвижимостью    10774
операции с автомобилем       4290
получение образования        3998
проведение свадьбы           2315
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Сначала сгруппируем данные до количеству детей и посмотрим, есть ли зависимость между количиством детей и наличием долга.

In [34]:
data.groupby('children')['debt'].agg(['mean','count','sum'])

,mean,count,sum
children,,,
0,0.075263,14137,1064
1,0.092346,4808,444
2,0.094542,2052,194
3,0.081818,330,27
4,0.097561,41,4
5,0.000000,9,0


На первый взгляд очевидно, что чем меньше детей, тем больше долгов. Посмотрим сколько всего людей с детьми взяли кредиты.

In [35]:
data.groupby('children')['debt'].count()


children
0    14137
1     4808
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64

Бездетные взяли больше всего кредитов, поэтому у них и больше неплатежей.
Нужно посмотреть долю неплатежей относительно общего числа кредитов в группе. 
Среднее как раз показывает отношение неплатежей к общему числу кредитов. Посчитаем процент неплатежей по кредиту для каждой группы.

In [36]:
(data.groupby('children')['debt'].mean()*100).astype('int')

children
0    7
1    9
2    9
3    8
4    9
5    0
Name: debt, dtype: int32

Явной связи между количеством детей и неплатежами по кредиту нет. Те у кого 5 детей почти не берут кредиты, но платят вовремя. Семей с 5 детьми достаточно мало.

Посмотрим есть ли зависимость с семейным положением. Создадим сводную таблицу со средними значениями.

In [37]:
data_pivot = data.pivot_table(index=['family_status_id'],columns='children', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot =(data_pivot.rename(index = dict(family_status_df.values))*100).astype('int')
data_pivot

children,0,1,2,3,4,5
family_status_id,,,,,,
женат / замужем,6,8,9,6,10,0
гражданский брак,8,11,8,14,0,0
вдовец / вдова,6,9,15,0,0,0
в разводе,6,6,8,9,0,0
не женат / не замужем,9,11,12,12,50,0


Доля неплатежей среди неженатых с 4 детьми 50%. Скорее всего выборка в этой категории нерепрезентативная. Посмотрим, сколько всего человек брали кредит.

In [38]:
data_pivot = data.pivot_table(index=['family_status_id'],columns='children', values='debt', aggfunc='count',dropna=True, fill_value=0)
data_pivot =data_pivot.rename(index = dict(family_status_df.values))
data_pivot

children,0,1,2,3,4,5
family_status_id,,,,,,
женат / замужем,7497,2975,1533,249,29,7
гражданский брак,2734,995,343,56,8,2
вдовец / вдова,851,77,20,6,1,0
в разводе,788,312,81,11,1,0
не женат / не замужем,2267,449,75,8,2,0


Всего двое неженатых с 4 детьми взяли кредит и один из них не заплатил вовремя.

Посмотрим связь неплатежей с количеством детей и образованием.

In [39]:
data_pivot = data.pivot_table(index=['education_id'],columns='children', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = (data_pivot.rename(index=dict(education_df.values))*100).astype('int')
data_pivot

children,0,1,2,3,4,5
education_id,,,,,,
высшее,5,5,5,4,0,0
среднее,8,10,10,9,12,0
неоконченное высшее,7,11,12,25,0,0
начальное,7,16,24,0,0,0
ученая степень,0,0,0,0,0,0


Наибольший процент неплатежей у людей c неоконченным высшим образованием и 3 детьми. Примерно такой же процент неплатежей у людей с начальным образованием и 2 детьми. У людей со средним и высшим образованием процент неплатежей практически не зависит от количества детей и составляет ~ 10% и ~ 5 % соответственно.Кандидаты и доктора наук платят по кредитам вовремя, но и кредитов они берут мало.

Создадим сводную таблицу с целью кредита и количеством детей.

In [40]:
data_pivot = data.pivot_table(index=['purpose_category'],columns='children', values='debt', aggfunc='mean',dropna=True, fill_value=0)
(data_pivot*100).astype('int')

children,0,1,2,3,4,5
purpose_category,,,,,,
операции с автомобилем,8,10,12,8,10,0
операции с недвижимостью,6,8,8,7,14,0
получение образования,8,10,11,5,0,0
проведение свадьбы,7,9,5,15,0,0


15% неплатежей у людей с 3 детьми, которые берут кредит на свадьбу. 14% неплатежей у людей с 4 детьми которые берут кредит на недвижимость.

Посмотрим на уровень дохода, тип занятости, количество детей и наличие задолженности.

In [41]:
data_pivot = data.pivot_table(index=['total_income_category'],columns='children', values='debt', aggfunc='mean',dropna=True, fill_value=0)
(data_pivot*100).astype('int')

children,0,1,2,3,4,5
total_income_category,,,,,,
A,8,11,0,0,0,0
B,6,7,8,4,14,0
C,7,9,9,9,8,0
D,4,6,11,20,0,0
E,11,0,0,0,0,0


In [42]:
data_pivot = data.pivot_table(index=['total_income_category'],columns='children', values='debt', aggfunc='count',dropna=True, fill_value=0)
data_pivot

children,0,1,2,3,4,5
total_income_category,,,,,,
A,12,9,3,1,0,0
B,3252,1167,502,88,7,4
C,10612,3562,1512,236,34,4
D,244,66,34,5,0,1
E,17,4,1,0,0,0


В категории дохода D (ниже среднего) чем больше детей, тем больше неплатежей. В других категориях однозначной зависимости нет. Но количество кредитов в этой категории у людей с 3 детьми небольшое, выборка нерепрезентативная, поэтому 20% неплатежей это недостоверный результат.

Посмотрим на тип доходов, наличие детей и количество неплатежей.

In [43]:
data_pivot = data.pivot_table(index=['income_type'],columns='children', values='debt', aggfunc=('mean'),dropna=True, fill_value=0)
(data_pivot*100).astype('int')

children,0,1,2,3,4,5
income_type,,,,,,
безработный,0,100,0,0,0,0
в декрете,0,0,100,0,0,0
госслужащий,6,5,3,5,0,0
компаньон,7,8,6,6,0,0
пенсионер,5,5,15,16,0,0
предприниматель,0,0,0,0,0,0
сотрудник,8,10,11,9,12,0
студент,0,0,0,0,0,0


У пенсионеров, чем больше детей, тем больше процент неплатежей. Наблюдаются большой процент просрочек у безработных и тех, кто в декрете, но однозначно сказать, что жто связано с количеством детей нельзя, там очень маленька выборка.

##### Вывод 1: Зависимость между количеством детей и возвратом кредита в срок.
- Больше всего кредитов берут бездетные люди, поэтому по ним и больше всего задержек с платежами. Правильнее сравнивать процент невозвратов по каждой группе. Проблема в том, что при увеличении количесва детей выборка может стать нерепрезентативной.
- Однозначная зависимость между количеством детей и просрочками по кредитам отсктствует.
- У одиноких граждан, а так же граждан с неоконченным высшим или начальным образованием и пенсионеров при увеличении количества детей возрастает процент невозвратов по кердитам.
- Процент неплатежей выше по кредитам на свадьбу у людей с 3 детьми и по кредитам на недвижимость у людей с 4 детьми.


##### Вопрос 2:  Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем данные по семейному положению и посчитаем среднее по долгу.Чтобы было удобнее смотреть, переведем все в проценты.

In [44]:
(data.groupby('family_status_id')['debt'].mean()*100).astype('int')

family_status_id
0    7
1    9
2    6
3    7
4    9
Name: debt, dtype: int32

In [45]:
data.groupby('family_status_id')['debt'].sum()

family_status_id
0    928
1    385
2     63
3     84
4    273
Name: debt, dtype: int64

In [46]:
data.groupby('family_status_id')['debt'].count()

family_status_id
0    12290
1     4138
2      955
3     1193
4     2801
Name: debt, dtype: int64

Связь между семейным положением и процентам неплатежей отсутсвует. Хотя у тех, кто живет гражданским браком и неженатых процент неплатежей немного выше.

Посмотрим на разные сочетания семейного положения и других параметров. С количеством детей мы уже определились выше. Посмотрим на образование.

In [47]:
data_pivot = data.pivot_table(index=['family_status_id'],columns='education_id', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = data_pivot.rename(index = dict(family_status_df.values))
data_pivot = (data_pivot.rename(columns=dict(education_df.values))*100).astype('int')
data_pivot

education_id,высшее,среднее,неоконченное высшее,начальное,ученая степень
family_status_id,,,,,
женат / замужем,5,8,8,10,0
гражданский брак,5,10,8,19,0
вдовец / вдова,5,6,15,0,0
в разводе,4,7,12,10,0
не женат / не замужем,5,11,9,8,0


Наибольший процент неплатежей у тех, кто живет гражданским браком и имеет начальное образование, а так же у овдовевших и разведенных с неоконченным высшим.

Посмотрим на семейный статус и цель кредита.

In [48]:
data_pivot = data.pivot_table(index=['family_status_id'],columns='purpose_category', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = data_pivot.rename(index = dict(family_status_df.values))
data_pivot = (data_pivot.rename(columns=dict(education_df.values))*100).astype('int')
data_pivot

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
family_status_id,,,,
женат / замужем,8,6,8,0
гражданский брак,11,9,14,7
вдовец / вдова,9,5,7,0
в разводе,7,6,7,0
не женат / не замужем,12,8,10,0


По всем категориям кредитов процент невозвратов немного выше у тех, кто живет гражданским браком и у неженатых.

Посмотрим семейный статус и категорию дохода.

In [49]:
data_pivot = data.pivot_table(index=['family_status_id'],columns='income_type', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = (data_pivot.rename(index = dict(family_status_df.values))*100).astype('int')
data_pivot

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
family_status_id,,,,,,,,
женат / замужем,100,100,5,6,5,0,8,0
гражданский брак,0,0,6,9,5,0,10,0
вдовец / вдова,0,0,4,9,6,0,5,0
в разводе,0,0,4,2,5,0,9,0
не женат / не замужем,0,0,7,8,4,0,11,0


Связь между семейным положением, категорией дохода и неплатежами отсутсвует. Выборка по безработным, декретникам, предпренимателям и студентам нерепрезентативная. Видимо им кредиты не дают.

In [50]:
data_pivot = data.pivot_table(index=['family_status_id'],columns='total_income_category', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = (data_pivot.rename(index = dict(family_status_df.values))*100).astype('int')
data_pivot

total_income_category,A,B,C,D,E
family_status_id,,,,,
женат / замужем,7,6,7,5,14
гражданский брак,16,6,10,8,0
вдовец / вдова,0,5,6,0,0
в разводе,0,6,7,11,0
не женат / не замужем,0,9,9,8,0


Видимо самые богатые не берут кредиты, а самым бедным не дают, поэтому колонки `A` и `E` не являются репрезентативными. Связь между уровнем дохода, семейным положением и долгами отсутсвует.

##### Вывод 2: Зависимость между семейным положением и возвратом кредита в срок.
- Зависимость между семейным положением и неплатежами по кредитам отсутствует.
- Если смотреть по целям кредита, то у тех, кто живет гражданским браком или неженат процент невозвратов по всем категорям кредитов немного выше, чем у других.
- У тех, кто живет гражданским браком и начальное образование, процент невозвратов достаточно высокий ~19%. А так же у овдовевших и разведенных с неоконченным высшим процент невозвратов выше, чем у остальных.

##### Вопрос 3:  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посмотрим на долю невозвратов, количество невозвратов и общее количество кредитов по категорям дохода.

In [51]:
(data.groupby('total_income_category')['debt'].mean()*100).astype('int')

total_income_category
A    8
B    7
C    8
D    6
E    9
Name: debt, dtype: int32

In [52]:
data.groupby('total_income_category')['debt'].sum()

total_income_category
A       2
B     354
C    1354
D      21
E       2
Name: debt, dtype: int64

In [53]:
data.groupby('total_income_category')['debt'].count()

total_income_category
A       25
B     5020
C    15960
D      350
E       22
Name: debt, dtype: int64

Связь между уровнем дохода и неплатежами отсутсвует. Хоятя у самых бедных процент невозвратов немного выше.

In [54]:
data_pivot = data.pivot_table(index=['total_income_category'],columns='education_id', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = (data_pivot.rename(columns = dict(education_df.values))*100).astype('int')
data_pivot

education_id,высшее,среднее,неоконченное высшее,начальное,ученая степень
total_income_category,,,,,
A,5,14,0,0,0
B,4,8,12,10,0
C,5,9,7,11,0
D,3,6,0,0,0
E,0,9,0,0,0


Процент неплатежей немного выше у людей с средним `C` или высоким доходом `B` и начальным образованием и высоким доходом `B` и неоконченным высшим образованием.

In [55]:
data_pivot = data.pivot_table(index=['total_income_category'],columns='purpose_category', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = (data_pivot.rename(columns = dict(education_df.values))*100).astype('int')
data_pivot

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
total_income_category,,,,
A,0,5,25,0
B,8,6,7,5
C,9,7,9,8
D,13,3,2,8
E,0,18,0,0


Связь между уровнем дохода, целью крудита и долгами отсутсвует. В строках `A` и `E` мало значений, поэтому они нерепрезентативные. Граждане с низким доходом `D` чаще не платят вовремя за автомобильные кредиты.

In [56]:
data_pivot = data.pivot_table(index=['total_income_category'],columns='income_type', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = (data_pivot.rename(columns = dict(education_df.values))*100).astype('int')
data_pivot

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
total_income_category,,,,,,,,
A,0,0,0,13,0,0,0,0
B,0,0,4,6,5,0,8,0
C,100,100,6,7,5,0,9,0
D,0,0,4,4,4,0,8,0
E,0,0,0,0,13,0,0,0


Связь между уровнем дохода, местом работы и долгами отсутсвует. В строках A и E мало значений, поэтому они нерепрезентативные. В колонках `безработный` и `в декрете` тоже мало значений, они тоже нерепрезентативные. Бедные пенсионеры и богатые компаньены одинаково часто неплатят по кредитам.

##### Вывод 3: Зависимость между уровнем дохода и возвратом кредита в срок.
- Зависимость между уровнем дохода и неплатежами отсутсвует. Видимо размер кредита зависит от уровня дохода, поэтому процент неплатежей примерно одинаковый в разных категорях дохода.
- Категори с самым высоким и самым низким доходом берут мало кредитов, и выборки в этих категориях не такие репрезентативные как в других категориях.
- В категории с низким доходом `D` чаще не платят за кредиты по автомобилям.

##### Вопро 4: Как разные цели кредита влияют на его возврат в срок?

In [57]:
(data.groupby('purpose_category')['debt'].mean()*100).astype('int')

purpose_category
операции с автомобилем      9
операции с недвижимостью    7
получение образования       9
проведение свадьбы          7
Name: debt, dtype: int32

In [58]:
data.groupby('purpose_category')['debt'].sum()

purpose_category
операции с автомобилем      401
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64

In [59]:
data.groupby('purpose_category')['debt'].count()

purpose_category
операции с автомобилем       4290
операции с недвижимостью    10774
получение образования        3998
проведение свадьбы           2315
Name: debt, dtype: int64

Возврат кредита в срок не зависит от целей крудита.

In [60]:
data_pivot = data.pivot_table(index=['purpose_category'],columns='education_id', values='debt', aggfunc='mean',dropna=True, fill_value=0)
data_pivot = (data_pivot.rename(columns = dict(education_df.values))*100).astype('int')
data_pivot

education_id,высшее,среднее,неоконченное высшее,начальное,ученая степень
purpose_category,,,,,
операции с автомобилем,6,10,10,10,0
операции с недвижимостью,4,7,8,11,0
получение образования,5,10,9,9,0
проведение свадьбы,4,9,7,14,0


Чаще всего не платят по кредитам на свадьбу люди с начальным образованием.

In [61]:
data_pivot = data.pivot_table(index=['purpose_category'],columns='income_type', values='debt', aggfunc='mean',dropna=True, fill_value=0)
(data_pivot*100).astype('int')

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
purpose_category,,,,,,,,
операции с автомобилем,0,100,7,8,6,0,11,0
операции с недвижимостью,50,0,4,6,4,0,8,0
получение образования,0,0,8,7,6,0,11,0
проведение свадьбы,0,0,4,9,5,0,8,0


Связь между целью кредита, местом работы и неплатежами отсутсвует. По безработным и декретным выборка нерепрезентативная.

##### Вывод 4: Как разные цели кредита влияют на его возврат в срок?
- Цели кредита не влияют на его возврат в срок.
- Наибольший процент неплатежей среди кредитов на свадьбу у людей с начальным образованием.


## Общий вывод:

**- Есть ли зависимость между количеством детей и возвратом кредита в срок?**
 - Больше всего кредитов берут женатые без детей, соответсвенно в этой же категории больше всего невозвратов. Сравнивать абсолютные значения между категориями некорректно, поэтому правильнее смотреть на долю невозвратов в каждой категории.
 - Связь между количеством детей и процентом невозвратов кретита в срок отсутсвуюет.
 - У одиноких граждан, граждан с неоконченным высшим или начальным образованием, пенсионеров c большим количеством детей процент неплатежей выше, чем у остальных.
 - Доля неплатежей по кредитам на свадьбу у граждан с 3 детьми и по кредитам на недвижимость у граждан с 4 детьми выше, чем у остальных. Но таких кредитов достаточно мало, поэтому выборка по этим категориям может быть нерепрезентативной.
 
**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**
 - Больше всего кредитов берут женатые, соответственно и неплатежей больше среди женатых. 
 - Связь между долей неплатежей и семейным статусом отсутсвует. Хотя у тех кто живет гражданским браком и у нежинатых процент неплатежей немного выше.
 - Наибольший процент неплатежей у живущих гражданским браком с начальным образованием. Так же высокая доля неплатежей у разведенных и вдовцов с неоконченным высшим.
 - Наибольший процент неплатежей по кредитам на образование у тех, кто живет гражданским браком.
 
**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**
 - Самые богатые и самые бедные категории граждан реже берут кредиты, поэтому информации в этих группах меньше, чем в остальных и она может быть нерепрезентативной.
 - Больше всего кредитов берут граждани со средним доходом или с доходом выше среднего, соответсвенно больше всего неплатежей в этих же категориях.
 - Однозначная связь между уровнем дохода и долей неплатежей по группам дохода отсутсвует.
 - Процент неплатежей немного выше у самой бедной группы.
 - Бедные пенсионеры и богатые компаньены одинаково часто не платят во время по кредитам, доля неплатежей выше, чем у остальных групп.
 - Граждане с доходом ниже среднего чаще всего не платят за автомобильные кредиты.
 
**- Как разные цели кредита влияют на его возврат в срок?**

   - Больше всего кредитов берут на недвижимость, соответственно в этой категории и больше всего неплатежей.
   - Однозначная связь между целью кредита и долей неплатежей отсутсвует.
   - По кредитам на свадьбу и на автомобиль процент неплатежей немного выше, чем по остальным категориям.
   - Чаще всего по кредитам на свадьбу не платят люди с начальным образованием.
